# BEE 4750 Homework 5: Solid Waste Disposal

**Name**: Akshara Chandrabalan

**ID**: ac2837

> **Due Date**
>
> Friday, 11/10/23, 9:00pm

## Overview

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

``` julia
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
```

``` julia
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables
```

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Downloads/hw05-akshara-c`


In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Background

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided in the table below.

|    **Disposal Facility**     | **Capacity** (Mg/d) | **Fixed cost** (\$/d) | **Tipping Fee ** (\$/Mg) | **Recycling Cost** (\$/Mg) |
|:-------------:|:-------------:|:-------------:|:----------:|:--------------:|
|           Landfill           |         200         |         2000          |            50            |                            |
| Materials Recycling Facility |         350         |         1500          |            7             |    40 (per Mg recycled)    |
|   Waste-to-Energy Facility   |         210         |         2500          |            60            |                            |

Transportation costs are \$1.5/Mg-km, and the relative distances between
the cities and facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

|   **Component**   | **% of total mass** | **Combustion ash** (%) | **MRF Recycling rate** (%) |
|:---------------------:|:--------------:|:---------------:|:---------------:|
|    Food Wastes    |         15          |           8            |             0              |
| Paper & Cardboard |         40          |           7            |             55             |
|     Plastics      |          5          |           5            |             15             |
|     Textiles      |          3          |           10           |             10             |
|  Rubber, Leather  |          2          |           15           |             0              |
|       Wood        |          5          |           2            |             30             |
|    Yard Wastes    |         18          |           2            |             40             |
|       Glass       |          4          |          100           |             60             |
|      Ferrous      |          2          |          100           |             75             |
|     Aluminum      |          2          |          100           |             80             |
|    Other Metal    |          1          |          100           |             50             |
|   Miscellaneous   |          3          |           70           |             0              |

The information in the above table will help you determine the overall
recycling and ash fractions. Note that the recycling residuals, which
may be sent to either landfill or the WTE, have different ash content
than the ash content of the original MSW. You will need to determine
these fractions to construct your mass balance constraints.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

## Problems (Total: 40 Points)

### Problem 1 (22 points)

In this problem, you will develop an optimal disposal plan for the two
cities.

#### Problem 1.1 (3 points) 

Based on the information above, calculate the overall recycling and ash
fractions for the waste produced by each city.

ANSWER: \
overall recycling fraction = (0.15 * 0) + (0.4 * 0.55) + (0.05 * 0.15) + (0.03 * 0.1) + (0.02 * 0) + (0.05 * 0.3) + (0.18 * 0.4) + (0.04 * 0.6) + (0.02 * 0.75) + (0.02 * 0.8) + (0.01 * 0.5) + (0.03 * 0)

overall recycling fraction = 0.38 (38%)

overall ash fraction = (0.15 * 0.08) + (0.4 * 0.07) + (0.05 * 0.05) + (0.03 * 0.1) + (0.02 * 0.15) + (0.05 * 0.02) + (0.18 * 0.02) + (0.04 * 1) + (0.02 * 1) + (0.02 * 1) + (0.01 * 1) + (0.03 * 0.7)

overall ash fraction = 0.16 (16%)

#### Problem 1.2 (2 points)

What are the decision variables for your optimization problem? Provide
notation and variable meaning.

ANSWER: \
$W_{ij}$, in Mg/day, is the amount of waste transported from each city $i \in I$ to a given disposal site $j \in J$ where $I$ = [1:3] and $J$ = [1:3] (J[1] corresponds to LF, J[2] corresponds to MRF and J[3] corresponds to WTE)

$R_{kj}$, in Mg/day, is the amount of residual waste transported from disposal $k \in K$ to disposal $j$, where K = [2:3]

$Y_j$ is the operational (on/off) status of disposal j - binary and unitless


#### Problem 1.3 (3 points)

Formulate the objective function. Make sure to include any needed
derivations or justifications for your equation(s).

ANSWER: The objective is to minimize total cost, in dollars. Total cost is made up of transportation costs plus disposal costs.

Transportation costs: \
$a_{ij}$ is the cost of transporting waste from source i to disposal j in dollars/Mg-km \
$l_{ij}$ is the distance between source i and disposal j in km \
Therefore,
$$transportation \ costs = \sum_{i \in I, j \in J, k \in K} a_{ij} l_{ij} W_{ij} R_{kj}$$

Disposal costs: \
$c_j$ is the fixed cost of operating disposal j in dollars/day \
$b_j$ is the tipping fee (variable cost) in dolars/Mg \
Therefore,
$$disposal \ cost = \sum_{j \in J} [c_j + b_j[\sum_{i \in I} W_{ij} + \sum_{k \in K} R_{kj}]]$$

So, the objective function is as follows:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} \sum_{i \in I, j \in J, k \in K} a_{ij} l_{ij} W_{ij} R_{kj} + \sum_{j \in J} [c_jY_j + b_j\sum_{i \in I, k \in K} [W_{ij} + R_{kj}]]
$$

Process of simplifying: \
$LF \ cost = 2000Y_1 + 50(W_{11} + W_{21} + W_{31} + R_{21} + R_{31})$ \
$MRF \ cost = 1500Y_2 + 7(W_{12} + W_{22} + W_{32}) + 0.38(40)(W_{12} + W_{22} + W_{32})$ \
$WTE \ cost = 2500Y_3 + 60(W_{13} + W_{23} + W_{33} + R_{23})$ \
$total \ transportation \ cost = 1.5(5W_{11} + 15W_{21} + 13W_{31} + 32R_{21} + 18R_{31} + 30W_{12} + 25W_{22} + 45W_{32} + 15W_{13} + 10W_{23} + 20W_{33} + 15R_{23})$

$$
\min_{W_{ij}, Y_{j}, R_{kj}} 2000Y_1 + 50W_{11} + 50W_{21} + 50W_{31} + 50R_{21} + 50R_{31} + 1500Y_2 + 7W_{12} + 7W_{22} + 7W_{32} + 15.2W_{12} + 15.2W_{22} + 15.2W_{32} + 2500Y_3 + 60W_{13} \\ + 60W_{23} + 60W_{33} + 60R_{23} + 7.5W_{11} + 22.5W_{21} + 19.5W_{31} + 48R_{21} + 27R_{31} + 45W_{12} + 37.5W_{22} + 67.5W_{32} + 22.5W_{13} + 15W_{23} + 30W_{33} + 22.5R_{23}
$$ 

The simplified objective function to minimize cost (in dollars) is:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} 57.5W_{11} + 67.2W_{12} + 82.5W_{13} + 72.5W_{21} + 59.7W_{22} + 75W_{23} + 69.5W_{31} + 89.7W_{32} + 90W_{33} + 98R_{21} + 77R_{31} + \\ 82.5R_{23} + 2000Y_1 + 1500Y_2 + 2500Y_3 
$$ 


#### Problem 1.4 (4 points) 

Derive all relevant constraints. Make sure to include any needed
justifications or derivations.

ANSWER: \
City mass-balance constraints:
$$W_{11} + W_{12} + W_{13} = 100 \ Mg/d$$
$$W_{21} + W_{22} + W_{23} = 90 \ Mg/d$$
$$W_{31} + W_{32} + W_{33} = 120 \ Mg/d$$

Residual mass-balance constraints:
$$R_{31} = 0.16(W_{13} + W_{23} + W_{33} + R_{23}) \ (for \ WTE)$$
$$R_{21} + R_{23} = 0.62(W_{12} + W_{22} + W_{32}) \ (for \ MRF)$$

Disposal limit constraints:
$$W_{11} + W_{21} + W_{31} + R_{21} + R_{31} \leq 200 \ Mg/d \ (for \ LF)$$ 
$$W_{12} + W_{22} + W_{32} \leq 350 \ Mg/d \ (for \ MRF)$$
$$W_{13} + W_{23} + W_{33} + R_{23} \leq 210 \ Mg/d \ (for \ WTE)$$ 

Commitment and non-negativity constraints: LF is always on, so,
$$Y_1 = 1$$

MRF and WTE can be turned on and off, so,
$$Y_2 =
\begin{cases}
    0 & \text{if } W_{12} + W_{22} + W_{32} = 0 \\
    1 & \text{else}
\end{cases}
$$
$$Y_3 =
\begin{cases}
    0 & \text{if } W_{13} + W_{23} + W_{33} + R_{23} = 0 \\
    1 & \text{else}
\end{cases}
$$

Amount of waste and residual cannot be negative, so,
$$W_{ij}, R_{kj} \geq 0$$



#### Problem 1.5 (3 points) 

Implement your optimization problem in `JuMP`.

In [3]:
MSW_disposal_model = Model(HiGHS.Optimizer)

# definition of sets
I = 1:3 # city
J = 1:3 # disposal site where 1 corresponds to LF, 2 corresponds to MRF and 3 corresponds to WTE
# disposal site from which residual waste is transported - no residual waste transported from LF, 
# but range starts at 1 to maintain a 3 cell vector
K = 1:3

# decision variables definition + non-negativity 
@variable(MSW_disposal_model, W[i in I, j in J] >= 0)
@variable(MSW_disposal_model, R[k in K, j in J] >= 0)
@variable(MSW_disposal_model, Y[j in J], Bin) # binary variable where value is either 0 or 1 

# city mass-balance constraints
@constraint(MSW_disposal_model, city1_mass, sum(W[1, j] for j in J) == 100)
@constraint(MSW_disposal_model, city2_mass, sum(W[2, j] for j in J) == 90)
@constraint(MSW_disposal_model, city3_mass, sum(W[3, j] for j in J) == 120)
# residual mass-balance constraints
@constraint(MSW_disposal_model, WTE_residual, 0.16 * (W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3]) == R[3, 1])
@constraint(MSW_disposal_model, MRF_residual, 0.62 * (W[1, 2] + W[2, 2] + W[3, 2]) == R[2, 1] + R[2, 3])
# disposal limit constraints 
@constraint(MSW_disposal_model, LF_disp_limit, W[1, 1] + W[2, 1] + W[3, 1] + R[2, 1] + R[3, 1] <= 200)
@constraint(MSW_disposal_model, MRF_disp_limit, W[1, 2] + W[2, 2] + W[3, 2] <= 350)
@constraint(MSW_disposal_model, WTE_disp_limit, W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3] <= 210)
# commitment constraints
M = 1500 # choosing a large M for the "big-M" reformulation
@constraint(MSW_disposal_model, commitLF, Y[1] == 1)
@constraint(MSW_disposal_model, commitMRF, M * Y[2] >= W[1, 2] + W[2, 2] + W[3, 2])
@constraint(MSW_disposal_model, commitWTE, M * Y[3] >= W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3])

@objective(MSW_disposal_model, Min, 57.5W[1, 1] + 67.2W[1, 2] + 82.5W[1, 3] + 72.5W[2, 1] + 59.7W[2, 2] + 75W[2, 3] + 69.5W[3, 1] + 89.7W[3, 2] +
                                    90W[3, 3] + 98R[2, 1] + 77R[3, 1] + 82.5R[2, 3] + 2000Y[1] + 1500Y[2] + 2500Y[3]);

#### Problem 1.6 (2 points)

Find the optimal solution. Report the optimal objective value.

In [4]:
optimize!(MSW_disposal_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
9 rows, 14 cols, 37 nonzeros
8 rows, 13 cols, 35 nonzeros

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   2000            inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   2000            27792.857143      92.80%        0      0      0         0     0.1s
         0       0         0   0.00%   25944.750716    27792.857143       6.65%        0      0      0         6     0.1s

Solving report
  Status            Optimal
  Primal bound      27792.8571429
  Dual bound        27792.8571429
  Gap            

In [5]:
LF_status = Int(value.(Y[1])) # using Int to round to a whole number
@show LF_status
MRF_status = Int(value.(Y[2]))
@show MRF_status
WTE_status = Int(value.(Y[3]))
@show WTE_status

println()

W_11 = value.(W[1,1])
@show W_11
W_12 = Int(value.(W[1,2]))
@show W_12
W_13 = Int(value.(W[1,3]))
@show W_13
W_21 = Int(value.(W[2,1]))
@show W_21
W_22 = Int(value.(W[2,2]))
@show W_22
W_23 = value.(W[2,3])
@show W_23
W_31 = round(value.(W[3,1]), digits = 1)
@show W_31
W_32 = Int(value.(W[3,2]))
@show W_32
W_33 = round(value.(W[3,3]), digits = 1)
@show W_33;

println()

R_31 = round(value.(R[3,1]), digits = 1)
@show R_31
R_21 = Int(value.(R[2,1]))
@show R_21
R_23 = Int(value.(R[2,3]))
@show R_23;



LF_status = 1
MRF_status = 0
WTE_status = 1

W_11 = 100.0
W_12 = 0
W_13 = 0
W_21 = 0
W_22 = 0
W_23 = 90.0
W_31 = 79.0
W_32 = 0
W_33 = 41.0

R_31 = 21.0
R_21 = 0
R_23 = 0


ANSWER: The optimal objective value (minimized cost) is about 27,792.86 dollars. The associated waste disposal plan is as follows:

- only the LF and WTE disposal site is operational (MRF is not operating)
- $W_{11} = 100$  Mg/d, $W_{23} = 90$ Mg/d, $W_{31} = 79$ Mg/d, $W_{33} = 41$ Mg/d
- $R_{31} = 21$ Mg/d

#### Problem 1.7 (5 points) 

Draw a diagram showing the flows of waste between the cities and the
facilities. Which facilities (if any) will not be used? Does this
solution make sense?

ANSWER: 

![diagram](image.png)

The MRF facility will not be used. This makes sense because due to the MRF being located notably farther than the other two facilities, the transportation costs are high, and there would be the additional costs that arise from handling residual waste from the MRF. Another check that confirms the solution makes sense is that for each city, the sum of the waste being sent to disposal sites equals the amount of waste generated per day in each city. For example, for City 3, 79 Mg/d is sent to LF and 41 Mg/d is sent to WTE. The total waste disposed is 120 Mg/d, which is equal to the amount of waste produced by City 3. Also, the residual waste from WTE to LF is supposed to be 16% of the incoming waste to WTE. According to this solution, the total amount of incoming waste to WTE is 131 Mg/d, and 16% of that is about 21 Mg/d, which is the solution for $R_{31}$.

### Problem 2 (18 points)

It is projected that in the near future the state will introduce a
carbon tax that will increase the cost for transportation and for
disposal by incineration. It is estimated that the additional costs will
be:

-   tipping fee for the WTE facility will increase to\$75/Mg; and
-   transportation costs will increase to \$2/Mg-km.

In this context, the cities are considering adding another landfill and
want to know if this would be cost-effective compared to using the
current facilities with the carbon tax. This landfill would have a
maximum capacity of 100 Mg/day and would be located with the following
distances from the existing sites (excluding LF1):

| **City/Facility** | **Distance to LF2 (km)** |
|:-----------------:|:------------------------:|
|         1         |            45            |
|         2         |            35            |
|         3         |            15            |
|        MRF        |            35            |
|        WTE        |            50            |

The fixed cost of operating this facility would be the same as the first
landfill, but the tipping cost would be increased to \$60/Mg-day.

#### Problem 2.1 (5 points) 

What changes are needed to your optimization program from Problem 1 for
this decision problem? Formulate any different variables, objectives,
and/or constraints.

ANSWER:

Including disposal site LF2 to decision variable: 

$W_{ij}$, in Mg/day, is the amount of waste transported from each city $i \in I$ to a given disposal site $j \in J$ where $I$ = [1:3] and $J$ = [1:4] (J[1] corresponds to LF1, J[2] corresponds to MRF and J[3] corresponds to WTE, and J[4] corresponds to LF2)

Editing cost components:

$LF2 \ cost = 2000Y_4 + 60(W_{14} + W_{24} + W_{34} + R_{24} + R_{34})$

$WTE \ cost = 2500Y_3 + 75(W_{13} + W_{23} + W_{33} + R_{23})$ 

$total \ transportation \ cost = 2[5W_{11} + 15W_{21} + 13W_{31} + 32R_{21} + 18R_{31} + 30W_{12} + 25W_{22} + 45W_{32} + 15W_{13} + 10W_{23} + 20W_{33} + 15R_{23} + 45W_{14} + 35W_{24} + 15W_{34} + 35R_{24} + 50R_{34}]$

Updated objective function, in dollars:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} 2000Y_1 + 50W_{11} + 50W_{21} + 50W_{31} + 50R_{21} + 50R_{31} + 1500Y_2 + 7W_{12} + 7W_{22} + 7W_{32} + 15.2W_{12} + 15.2W_{22} + 15.2W_{32} + 2500Y_3 + 75W_{13} + 75W_{23} \\ + 75W_{33} + 75R_{23} + 2000Y_4 + 60W_{14} + 60W_{24} + 60W_{34} + 60R_{24} + 60R_{34} + 10W_{11} + 30W_{21} + 26W_{31} + 64R_{21} + 36R_{31} + 60W_{12} \\ + 50W_{22} + 90W_{32} + 30W_{13} + 20W_{23} + 40W_{33} + 30R_{23} + 90W_{14} + 70W_{24} + 30W_{34} + 70R_{24} + 100R_{34} 
$$ 

Updated objective function, simplified, in dollars:
$$
\min_{W_{ij}, Y_{j}, R_{kj}} 60W_{11} + 80W_{21} + 76W_{31} + 82.2W_{12} + 72.2W_{22} + 112.2W_{32} + 105W_{13} + 95W_{23} + 115W_{33} + 150W_{14} + 130W_{24} + 90W_{34} + 114R_{21} \\ + 86R_{31} + 105R_{23} + 130R_{24} + 160R_{34} + 2000Y_1 + 1500Y_2 + 2500Y_3 + 2000Y_4   
$$ 

Updating/adding constraints with respect to LF2:  

City mass-balance constraints:
$$W_{11} + W_{12} + W_{13} + W_{14}  = 100 \ Mg/d$$
$$W_{21} + W_{22} + W_{23} + W_{24} = 90 \ Mg/d$$
$$W_{31} + W_{32} + W_{33} + W_{34} = 120 \ Mg/d$$

Residual mass-balance constraints:
$$R_{31} + R_{34} = 0.16(W_{13} + W_{23} + W_{33} + R_{23}) \ (for \ WTE)$$
$$R_{21} + R_{23} + R_{24} = 0.62(W_{12} + W_{22} + W_{32}) \ (for \ MRF)$$

Disposal limit constraints:
$$W_{11} + W_{21} + W_{31} + R_{21} + R_{31} \leq 200 \ Mg/d \ (for \ LF1)$$ 
$$W_{12} + W_{22} + W_{32} \leq 350 \ Mg/d \ (for \ MRF)$$
$$W_{13} + W_{23} + W_{33} + R_{23} \leq 210 \ Mg/d \ (for \ WTE)$$ 
$$W_{14} + W_{24} + W_{34} + R_{24} + R_{34} \leq 100 \ Mg/d \ (for \ LF2)$$ 

Commitment and non-negativity constraints: LF1 and LF2 are always on, so,
$$Y_1 = 1$$

MRF, WTE, and LF2 can be turned on and off, so,
$$Y_2 =
\begin{cases}
    0 & \text{if } W_{12} + W_{22} + W_{32} = 0 \\
    1 & \text{else}
\end{cases}
$$
$$Y_3 =
\begin{cases}
    0 & \text{if } W_{13} + W_{23} + W_{33} + R_{23} = 0 \\
    1 & \text{else}
\end{cases}
$$
$$Y_4 =
\begin{cases}
    0 & \text{if } W_{14} + W_{24} + W_{34} + R_{24} + R_{34} = 0 \\
    1 & \text{else}
\end{cases}
$$

Amount of waste and residual cannot be negative, so,
$$W_{ij}, R_{kj} \geq 0$$

#### Problem 2.2 (3 points) 

Implement the new optimization problem in `JuMP`.

In [6]:
MSW_disposal_model_carbon = Model(HiGHS.Optimizer)

# definition of sets
I = 1:3 # city
J = 1:4 # disposal site where 1 corresponds to LF1, 2 corresponds to MRF, 3 corresponds to WTE, and 4 to LF2
# disposal site from which residual waste is transported - no residual waste transported from LF1 and LF2, 
# but range starts at 1 to maintain a 4 cell vector
K = 1:4

# decision variables definition + non-negativity 
@variable(MSW_disposal_model_carbon, W[i in I, j in J] >= 0)
@variable(MSW_disposal_model_carbon, R[k in K, j in J] >= 0)
@variable(MSW_disposal_model_carbon, Y[j in J], Bin) # binary variable where value is either 0 or 1 

# city mass-balance constraints
@constraint(MSW_disposal_model_carbon, city1_mass, sum(W[1, j] for j in J) == 100)
@constraint(MSW_disposal_model_carbon, city2_mass, sum(W[2, j] for j in J) == 90)
@constraint(MSW_disposal_model_carbon, city3_mass, sum(W[3, j] for j in J) == 120)
# residual mass-balance constraints
@constraint(MSW_disposal_model_carbon, WTE_residual, 0.16 * (W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3]) == R[3, 1] + R[3, 4])
@constraint(MSW_disposal_model_carbon, MRF_residual, 0.62 * (W[1, 2] + W[2, 2] + W[3, 2]) == R[2, 1] + R[2, 3] + R[2, 4])
# disposal limit constraints 
@constraint(MSW_disposal_model_carbon, LF1_disp_limit, W[1, 1] + W[2, 1] + W[3, 1] + R[2, 1] + R[3, 1] <= 200)
@constraint(MSW_disposal_model_carbon, MRF_disp_limit, W[1, 2] + W[2, 2] + W[3, 2] <= 350)
@constraint(MSW_disposal_model_carbon, WTE_disp_limit, W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3] <= 210)
@constraint(MSW_disposal_model_carbon, LF2_disp_limit, W[1, 4] + W[2, 4] + W[3, 4] + R[2, 4] + R[3, 4] <= 100)
# commitment constraints
M = 1500 # choosing a large M for the "big-M" reformulation
@constraint(MSW_disposal_model_carbon, commitLF1, Y[1] == 1)
@constraint(MSW_disposal_model_carbon, commitMRF, M * Y[2] >= W[1, 2] + W[2, 2] + W[3, 2])
@constraint(MSW_disposal_model_carbon, commitWTE, M * Y[3] >= W[1, 3] + W[2, 3] + W[3, 3] + R[2, 3])
@constraint(MSW_disposal_model_carbon, commitLF2, M * Y[4] >= W[1, 4] + W[2, 4] + W[3, 4] + R[2, 4] + R[3, 4])

@objective(MSW_disposal_model_carbon, Min, 60W[1, 1] + 80W[2, 1] + 76W[3, 1] + 82.2W[1, 2] + 72.2W[2, 2] + 112.2W[3, 2] + 105W[1, 3] + 95W[2, 3] +
                                           115W[3, 3] + 150W[1, 4] + 130W[2, 4] + 90W[3, 4] + 114R[2, 1] + 86R[3, 1] + 105R[2, 3] + 130R[2, 4] + 
                                           160R[3, 4] + 2000Y[1] + 1500Y[2] + 2500Y[3] + 2000Y[4]);

#### Problem 2.3 (5 points)

Find the optimal solution and report the optimal objective value.
Provide a diagram showing the new waste flows.

In [7]:
optimize!(MSW_disposal_model_carbon)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 20 cols, 53 nonzeros
11 rows, 20 cols, 53 nonzeros

Solving MIP model with:
   11 rows
   20 cols (3 binary, 0 integer, 0 implied int., 17 continuous)
   53 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   2000            inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   2000            30562.380952      93.46%        0      0      0         0     0.0s
         0       0         0   0.00%   26544.298273    30562.380952      13.15%        0      0      0         8     0.0s
 S       0       0         0   0.00%   26544.298273    30562.380952      13.15%        9      1      0         8     0.0s

Sol

In [8]:
LF1_status = Int(value.(Y[1])) # using Int to round to a whole number
@show LF1_status
MRF_status = Int(value.(Y[2]))
@show MRF_status
WTE_status = Int(value.(Y[3]))
@show WTE_status
LF2_status = Int(round(value.(Y[4])))
@show LF2_status

println()

W_11 = value.(W[1,1])
@show W_11
W_12 = value.(W[1,2])
@show W_12
W_13 = value.(W[1,3])
@show W_13
W_14 = value.(W[1,4])
@show W_14
W_21 = round(value.(W[2,1]), digits = 1)
@show W_21
W_22 = Int(value.(W[2,2]))
@show W_22
W_23 = round(value.(W[2,3]), digits = 1)
@show W_23
W_24 = value.(W[2,4])
@show W_24
W_31 = value.(W[3,1])
@show W_31
W_32 = value.(W[3,2])
@show W_32
W_33 = value.(W[3,3])
@show W_33
W_34 = value.(W[3,4])
@show W_34

println()

R_31 = round(value.(R[3,1]), digits = 1)
@show R_31
R_21 = value.(R[2,1])
@show R_21
R_23 = value.(R[2,3])
@show R_23;
R_34 = value.(R[3,4])
@show R_34;
R_24 = value.(R[2,4])
@show R_24;

LF1_status = 1
MRF_status = 0
WTE_status = 1
LF2_status = 1

W_11 = 100.0
W_12 = 0.0
W_13 = 0.0
W_14 = 0.0
W_21 = 78.1
W_22 = 0
W_23 = 11.9
W_24 = 0.0
W_31 = 20.0
W_32 = 0.0
W_33 = 0.0
W_34 = 100.0

R_31 = 1.9
R_21 = 0.0
R_23 = 0.0
R_34 = 0.0
R_24 = 0.0


ANSWER: The optimal objective value (minimized cost) is about 30,562.38 dollars. The associated waste disposal plan is as follows:

- only the LF1, WTE, and LF2 disposal site is operational (MRF is not operating)  
- $W_{11} = 100$  Mg/d, $W_{21} = 78.1$ Mg/d, $W_{23} = 11.9$ Mg/d, $W_{31} = 20$ Mg/d, $W_{34} = 100$ Mg/d
- $R_{31} = 1.9$ Mg/d

![diagram](image2.png)

#### Problem 2.4 (5 points)

Would you recommend that the cities build the new landfill? Why or why
not? Your answer should be based on your analysis but can draw on other
considerations as appropriate or desired.

ANSWER: Yes, because the model solution shows the operational status of LF2 should be "on". Thus, under the carbon tax scenario, having the second landfill is a part of the MSW disposal plan that results in the optimized minimal cost. Additionally, given the price increase for WTE, it makes sense that we're significantly reducing the amount disposed at WTE and diverting the disposal to LF2. However, additional considerations, such as the ecological cost associated with converting land to a landfill and stakeholder/public buy-in, should be taken into account in making the decision.  

## References

List any external references consulted, including classmates.

- https://ctan.math.washington.edu/tex-archive/info/undergradmath/undergradmath.pdf

- https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/

- https://docs.julialang.org/en/v1/stdlib/Markdown/

- BEE 4750/5750 - Lecture 21 Waste Management and Network Models

Consulted with Christine Swanson. 